# Hyperparameter Tuning

<li> Install skopt library

In [2]:
pip install scikit-optimize

     |████████████████████████████████| 100 kB 5.7 MB/s ta 0:00:01
     |████████████████████████████████| 189 kB 40.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


<li> Manual Tuning - Super Slow

In [14]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

wine = load_wine()

#print(wine.DESCR)

x, y = wine.data, wine.target

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.33, random_state=32)


#Scale feature values to between -1 and 1 but retain spacial orientation of data
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_train)

#apply scaling to both train and test set
x_train = scaling.transform(x_train)
x_test = scaling.transform(x_test)

model = SVC()
print('-'*30)
print('Just crossvalidation')
print('-'*30)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
m_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (np.mean(m_scores), np.std(m_scores)))


from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

# define the space of hyperparameters to search
search_space = list()
search_space.append(Real(1e-6, 1000.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf'], name='kernel'))
search_space.append(Integer(1, 3, name='degree'))
search_space.append(Real(1e-6, 10.0, 'log-uniform', name='gamma'))


@use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    model = SVC()
    model.set_params(**params)
    # calculate 5-fold cross validation
    result = cross_val_score(model, x_train, y_train, cv=cv, n_jobs=-1, scoring='accuracy')
    # calculate the mean of the scores
    estimate = np.mean(result)
    # convert from a maximizing score to a minimizing score
    return 1.0 - estimate




import warnings
warnings.filterwarnings('ignore')

print('-'*30)
print('Manual Tuning')
print('-'*30)
result = gp_minimize(evaluate_model, search_space)
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

------------------------------
Just crossvalidation
------------------------------
Accuracy: 0.978 (0.037)
------------------------------
Manual Tuning
------------------------------
Best Accuracy: 0.981
Best Parameters: [405.14576119775586, 'rbf', 2, 0.39568268352745994]


<li>Auto Tuning - Fast but not as Accurate

In [15]:
print('-'*30)
print('Auto Tuning')
print('-'*30)

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

search_space = dict()
search_space['C'] = (1e-6, 1000.0, 'log-uniform')
search_space['gamma'] = (1e-6, 10.0, 'log-uniform')
search_space['degree'] = (1,3)
search_space['kernel'] = [ 'linear', 'poly', 'rbf']

#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search

model = SVC()
search = GridSearchCV(model, search_space, n_jobs=-1, cv=cv,scoring='acuracy')

search.fit(x_train, y_train)
# report the best result
print(search.best_score_)
print(search.best_params_)

------------------------------
Auto Tuning
------------------------------
0.9606060606060606
{'C': 1000.0, 'degree': 1, 'gamma': 1e-06, 'kernel': 'linear'}
